In [35]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras as K 
import keras.layers as layers
from sklearn.model_selection import train_test_split

In [15]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv('/home/xiaotongfeng/Desktop/TensorFlow2-Examples/2_StructuredData_Classify/heart.csv')
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [16]:
train,test = train_test_split(dataframe,test_size = 0.2)
train,valid = train_test_split(train,test_size = 0.2)
train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
222,66,0,3,146,278,0,2,152,0,0.0,2,1,normal,0
263,65,1,4,120,177,0,0,140,0,0.4,1,0,reversible,0
203,67,0,4,106,223,0,0,142,0,0.3,1,2,normal,0
123,57,1,4,152,274,0,0,88,1,1.2,2,1,reversible,0
272,51,0,4,130,305,0,0,142,1,1.2,2,0,reversible,1


In [17]:
tf.__version__

'2.2.0'

95     1
144    0
47     0
191    0
301    1
      ..
171    0
275    0
51     0
26     0
204    0
Name: target, Length: 193, dtype: int64


In [18]:
train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
222,66,0,3,146,278,0,2,152,0,0.0,2,1,normal,0
263,65,1,4,120,177,0,0,140,0,0.4,1,0,reversible,0
203,67,0,4,106,223,0,0,142,0,0.3,1,2,normal,0
123,57,1,4,152,274,0,0,88,1,1.2,2,1,reversible,0
272,51,0,4,130,305,0,0,142,1,1.2,2,0,reversible,1


In [14]:
def df_to_dataset(dataframe,shuffle = True, batch_size = 32):
    dataframe_ = dataframe.copy()
    labels = dataframe_.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe_),labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe_))
    ds = ds.batch(batch_size)
    return ds

In [23]:
train_ds = df_to_dataset(train,batch_size=5)
val_ds = df_to_dataset(valid,batch_size=5)
test_ds = df_to_dataset(test,batch_size=5)
list1 = list(train_ds.as_numpy_iterator())
print(list1[0])

({'age': array([42, 38, 53, 59, 58]), 'sex': array([0, 1, 1, 1, 1]), 'cp': array([3, 1, 4, 3, 4]), 'trestbps': array([120, 120, 140, 126, 100]), 'chol': array([209, 231, 203, 218, 234]), 'fbs': array([0, 0, 1, 1, 0]), 'restecg': array([0, 0, 2, 0, 0]), 'thalach': array([173, 182, 155, 134, 156]), 'exang': array([0, 1, 1, 0, 0]), 'oldpeak': array([0. , 3.8, 3.1, 2.2, 0.1]), 'slope': array([2, 2, 3, 2, 1]), 'ca': array([0, 0, 0, 1, 1]), 'thal': array([b'normal', b'reversible', b'reversible', b'fixed', b'reversible'],
      dtype=object)}, array([0, 1, 0, 1, 1]))


In [27]:
for feature_batch, label_batch in train_ds.take(3):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of ages:', feature_batch['age'])
    print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([61 52 54 49 59], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([58 57 41 44 62], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 0 0 0 1], shape=(5,), dtype=int64)
Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([60 58 44 58 61], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 1 0 0 0], shape=(5,), dtype=int64)


## 特征列转换
## feature column

In [38]:
example_batch = next(iter(train_ds))[0]
print(example_batch)
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

{'age': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([55, 42, 58, 43, 59])>, 'sex': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 0, 1, 1, 1])>, 'cp': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([2, 3, 4, 4, 4])>, 'trestbps': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([130, 120, 150, 150, 138])>, 'chol': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([262, 209, 270, 247, 271])>, 'fbs': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>, 'restecg': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 2, 0, 2])>, 'thalach': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([155, 173, 111, 171, 182])>, 'exang': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 1, 0, 0])>, 'oldpeak': <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0. , 0. , 0.8, 1.5, 0. ])>, 'slope': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 2, 1, 1, 1])>, 'ca': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>, 'thal': <tf.Tensor: shape=(5,), 

## 1) Numeric column

In [39]:
from tensorflow import feature_column

age = feature_column.numeric_column("age")
demo(age)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[55.]
 [42.]
 [58.]
 [43.]
 [59.]]


## 2) Bucketized column

In [40]:
age_buckets = feature_column.bucketized_column(age,boundaries=[18,50,80])
demo(age_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


## 3) Categorical column

In [42]:
thal = feature_column.categorical_column_with_vocabulary_list('thal',['fixed','normal','reversible'])
thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


## 4) Embedding column

In [43]:
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[-0.28048638  0.08295303  0.41781107  0.16522782 -0.21135572 -0.21493904
   0.15591055  0.16471213]
 [-0.28048638  0.08295303  0.41781107  0.16522782 -0.21135572 -0.21493904
   0.15591055  0.16471213]
 [-0.26925907 -0.29685724 -0.49263385 -0.17231683 -0.02121721 -0.397148
  -0.24269478 -0.6620404 ]
 [-0.28048638  0.08295303  0.41781107  0.16522782 -0.21135572 -0.21493904
   0.15591055  0.16471213]
 [-0.28048638  0.08295303  0.41781107  0.16522782 -0.21135572 -0.21493904
   0.15591055  0.16471213]]


## 5) Hashed feature column

In [44]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
      'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 6) Crossed feature columns

In [45]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## For training

In [48]:
feature_columns = []
for iter_ in ['age','trestbps','chol','thalach','oldpeak','slope','ca']:
    feature_columns.append(feature_column.numeric_column(iter_))

age_buckets = feature_column.bucketized_column(age,boundaries=[18,25,30,35,40,45,50,55,60,65])
feature_columns.append(age_buckets)

thal = feature_column.categorical_column_with_vocabulary_list('thal',['fixed','normal','reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

thal_embedding = feature_column.embedding_column(thal,dimension=8)
feature_columns.append(thal_embedding)

crossed_feature = feature_column.crossed_column([age_buckets,thal],hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)


In [49]:
feature_layer = layers.DenseFeatures(feature_columns)

In [50]:
model = keras.Sequential([
    feature_layer,
    layers.Dense(128,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
]
)


In [53]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'],run_eagerly = True)
model.fit(train_ds,validation_data=val_ds,epochs=10)

Epoch 1/10
39/39 [==============================] - 1s 18ms/step - loss: 1.2767 - accuracy: 0.6684 - val_loss: 0.7695 - val_accuracy: 0.8163
Epoch 2/10
39/39 [==============================] - 1s 18ms/step - loss: 0.4927 - accuracy: 0.7979 - val_loss: 0.3934 - val_accuracy: 0.8163
Epoch 3/10
39/39 [==============================] - 1s 18ms/step - loss: 0.5058 - accuracy: 0.7824 - val_loss: 0.3984 - val_accuracy: 0.8163
Epoch 4/10
39/39 [==============================] - 1s 18ms/step - loss: 0.4453 - accuracy: 0.8031 - val_loss: 0.3788 - val_accuracy: 0.8367
Epoch 5/10
39/39 [==============================] - 1s 18ms/step - loss: 0.7608 - accuracy: 0.7150 - val_loss: 0.5051 - val_accuracy: 0.7347
Epoch 6/10
39/39 [==============================] - 1s 18ms/step - loss: 0.4562 - accuracy: 0.7876 - val_loss: 0.3899 - val_accuracy: 0.8571
Epoch 7/10
39/39 [==============================] - 1s 19ms/step - loss: 0.3865 - accuracy: 0.8446 - val_loss: 0.4143 - val_accuracy: 0.7959
Epoch 8/10
39

In [54]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy",accuracy)

13/13 [==============================] - 0s 11ms/step - loss: 0.4267 - accuracy: 0.7377
Accuracy 0.7377049326896667
